In [35]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

import zipfile
with zipfile.ZipFile('../input/plates.zip', 'r') as zip_obj:
   # Extract all the contents of zip file in current directory
   zip_obj.extractall('/kaggle/working/')
    
print('After zip extraction:')
print(os.listdir("/kaggle/working/"))

In [36]:
data_root = '/kaggle/working/plates/'
print(os.listdir(data_root))

## Ячейкой ниже мы из нашей папки train сделали еще 2 папки (train и valid).
## в папку valid закинули 1/6 всех файлов из train 

In [37]:
import shutil 
from tqdm import tqdm

train_dir = 'train'
val_dir = 'val'

class_names = ['cleaned', 'dirty']

for dir_name in [train_dir, val_dir]:
    for class_name in class_names:
        os.makedirs(os.path.join(dir_name, class_name), exist_ok=True)

for class_name in class_names:
    source_dir = os.path.join(data_root, 'train', class_name)
    for i, file_name in enumerate(tqdm(os.listdir(source_dir))):
        if i % 6 != 0:
            dest_dir = os.path.join(train_dir, class_name) 
        else:
            dest_dir = os.path.join(val_dir, class_name)
        shutil.copy(os.path.join(source_dir, file_name), os.path.join(dest_dir, file_name))

In [38]:
!ls

In [39]:
!ls train

 ## torchvision.datasets.ImageFolder
 ImageFolder -  итерируется по директориям, получает картинки, формирует из этих картинок пару (тензор и метка картинки) и их можно уже передавать в нейронную сеть. Метки это название папки, в которых лежат изображения.
 
Но если ImageFolder использовать без второго аргумента, то он вернет изображения, которые открыты через библиотеку PIL (Python Image Library) -- это будут не pytorch тензоры. И для этого нам нужно сделать некоторую трансформацию этих изображений, что бы получить тензоры

## transforms по валидации 

Будем делать три трансформации. Мы их вместе соединим с помощью объекта "transforms.Compose", то есть все трансформации будут подряд проделываться

1. Ужмём до размера 224 на 224. 
   ResNet, который мы используем -- инвариантен к размерам, но объекты для обучения должны быть одинакового размера. Мы выбрали 224 на 224 -- ибо это размер изображений из датасета ImageNet, на котором предобучен наш ResNet.
2. Превращаем изображение в тензор 
3. Нормировка изображений. Отнимаем и делим наши каналы именно на такие константы. Именно так был отнормирован изначальный ResNet, если посмотреть его документацию

## transforms по train

примерним аугментацию (способы из одной картинки сделать сразу много), есть намного больше видов аугментацию, пока ограничемся только этим. 
1. RandomResizedCrop - вырезает рандомные участки изображения и приводит их к размеру 224 на 224.
2. RandomHorizontalFlip - отзеркаливание относительно горизонтали
3. трансформация в тензор
4. нормировка

## torch.utils.data.DataLoader

DataLoader - объединит тензоры из изображений в батчи из таких изображений. Потому что нам хочется несколько изображений подавать в нейронку и считать некоторую лосс-функцию сразу по нескольким.

 num_workers -- это количество тредов (потоков), которые будут выполнять вот эти трансформации.

In [40]:
import torch
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import time
import copy

from torchvision import transforms, models
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = torchvision.datasets.ImageFolder(train_dir, train_transforms)
val_dataset = torchvision.datasets.ImageFolder(val_dir, val_transforms)

batch_size = 8
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=batch_size)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=batch_size)

In [41]:
len(train_dataloader), len(train_dataset) # 32 / 8 = 4

#### Можем посмотреть, как выглядит одно изображение из train_dataloader.
#### Что бы его визуализировать, нужно применить обратную нормирвку к нашему тензору.
#### .permute(1, 2, 0) - меняет каналы местами


In [42]:
X_batch, y_batch = next(iter(train_dataloader))
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
plt.imshow(X_batch[0].permute(1, 2, 0).numpy() * std + mean);

#### Все тоже самое, но в виде фкнкции. Полезно для того, что бы отследить результат аугментации. Например,на 3 карнтинки, у нас изображон в основном только стол, вместо тарелки и это не так хорошо, можно поменять параметр в RandomResizedCrop

In [43]:
def show_input(input_tensor, title=''):
    image = input_tensor.permute(1, 2, 0).numpy()
    image = std * image + mean
    plt.imshow(image.clip(0, 1))
    plt.title(title)
    plt.show()
    plt.pause(0.001)

X_batch, y_batch = next(iter(train_dataloader))

for x_item, y_item in zip(X_batch, y_batch):
    show_input(x_item, title=class_names[y_item])

In [44]:
def train_model(model, loss, optimizer, scheduler, num_epochs):
    for epoch in range(num_epochs):
        print('Epoch {}/{}:'.format(epoch, num_epochs - 1), flush=True)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                dataloader = train_dataloader
                scheduler.step() # для планировщика даем команду, что произшла 1 эпоха 
                """нужно для того, что бы зафиксировать слои нормализации и просто на всякий случай"""
                model.train()  # Set model to training mode, 
            else:
                dataloader = val_dataloader
                """что бы во время валидации не изменялась нейронка"""
                model.eval()   # Set model to evaluate mode

            running_loss = 0.
            running_acc = 0.

            # Iterate over data.
            for inputs, labels in tqdm(dataloader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad() # обнуляем градиент, что бы он не накапливался каждый раз

                # forward and backward. Активируем все градиенты(веса), которые не заморожены 
                with torch.set_grad_enabled(phase == 'train'):
                    """Считаем предикшн модели, это не вероятности, а просто аквации нейронов"""
                    preds = model(inputs) 
                    loss_value = loss(preds, labels)
                    preds_class = preds.argmax(dim=1) # берем нейрон с максимальной активацией

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss_value.backward()
                        optimizer.step()

                # statistics
                running_loss += loss_value.item()
                running_acc += (preds_class == labels.data).float().mean()

            epoch_loss = running_loss / len(dataloader)
            epoch_acc = running_acc / len(dataloader)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc), flush=True)

    return model

In [45]:
model.fc.in_features

In [46]:
"""В pytorch есть много разных ResNet. resnet18 -  "18" говорит о том, что там 18 слоёв 
pretrained = True - то есть нам нужны веса, полученные вследствие обучения этого ResNet, на датасете ImageNet.
"""
model = models.resnet18(pretrained=True)

"""отключить рассчет градиента для всех слоев сети
сеть хорошо предобучена и мы не хотим, что бы веса менялись во время обучения
"""
for param in model.parameters():
    param.requires_grad = False

"""меняем последний слой, вместо 1000 классов, как в оригинале, у нас будет 2
по умолчанию градиенты там всегда считаются, так что это единственный слой, который обучается
"""
model.fc = torch.nn.Linear(model.fc.in_features, 2)

"""переложили модель на видеокарту"""
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

loss = torch.nn.CrossEntropyLoss() # функция активации 
optimizer = torch.optim.Adam(model.parameters(), lr=1.0e-3) # метод оптимизации


"""некоторый планировщик, scheduler, который будет снижать шаг градиентного спуска во время обучения.
Затухание LR в 0,1 раза каждые 7 эпох
Есть некоторая эвристика, что, с течением обучения, нужно уменьшать шаг градиентного спуска."""
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [47]:
train_model(model, loss, optimizer, scheduler, num_epochs=100);

### как вы помните, ImageFolder -- он принимал на вход путь к папке, в которой есть некоторые папки с классами. У нас был путь к папке "train", а внутри папки "train" был cleaned и dirty, и к сожалению, ImageFolder не сможет вам обработать путь к папке, в которой уже сразу лежат изображения, а у нас в тестовой директории уже сразу лежат изображения. И поэтому нам нужно сделать некоторый хак, то есть скопировать всю папку в тест -- в папку "test/unknown".

In [48]:
test_dir = 'test'
shutil.copytree(os.path.join(data_root, 'test'), os.path.join(test_dir, 'unknown'))

### Вторая проблема - это то, что мы не знаем, какие ID, какие названия изображения у нас генерируется, когда мы просим у DataLoader -- "дай нам следующий батч". Они по алфавиту идут или по дате создания, или просто случайным образом -- непонятно. Поэтому нам нужно переписать немножко ImageFolder, чтобы он нам отдавал не просто tuple, с самим изображением и его меткой, а ещё, чтобы он отдавал имя, ну, либо -- путь к изображению.

In [49]:
class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path
    
test_dataset = ImageFolderWithPaths('/kaggle/working/test', val_transforms)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [50]:
test_dataset

### смотрим как наша модель отработала на test, сохраняя уже вероятности в test_predictions

In [51]:
model.eval() ## фиксируем модельку 

test_predictions = []
test_img_paths = []
for inputs, labels, paths in tqdm(test_dataloader): ### labels - всегда будет None
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.set_grad_enabled(False):
        preds = model(inputs)
    test_predictions.append(
        torch.nn.functional.softmax(preds, dim=1)[:,1].data.cpu().numpy())
    test_img_paths.extend(paths)
    
test_predictions = np.concatenate(test_predictions)

In [52]:
inputs, labels, paths = next(iter(test_dataloader))

for img, pred in zip(inputs, test_predictions):
    show_input(img, title=pred)

In [53]:
submission_df = pd.DataFrame.from_dict({'id': test_img_paths, 'label': test_predictions})

In [54]:
submission_df['label'] = submission_df['label'].map(lambda pred: 'dirty' if pred > 0.5 else 'cleaned')
submission_df['id'] = submission_df['id'].str.replace('/kaggle/working/test/unknown/', '')
submission_df['id'] = submission_df['id'].str.replace('.jpg', '')
submission_df.set_index('id', inplace=True)
submission_df.head(n=6)

In [55]:
submission_df.to_csv('submission.csv')

In [56]:
!rm -rf train val test